In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
#tqdm เป็นการสร้าง progree bar ของการวน loop

In [ ]:
#txn = pd.read_excel('D:/BAc/29 BingoBox/Transaction converting/1.Sales Detail.xlsx' , sheet = 'Export')
#m_item =  pd.read_excel('D:/BAc/29 BingoBox/Transaction converting/Item.xlsx' , sheet = 'Items')
#template = pd.read_excel('D:/BAc/29 BingoBox/Transaction converting/netsuite_template.xlsx' , sheet = ['template'])

txn = pd.read_excel('1.Sales Detail.xlsx' , sheet_name = 'Export')
m_item =  pd.read_excel('2.InternalID.xlsx' , sheet_name = 'Items')

#python อ่านไฟล์ 97 ไม่ได้ต้องเปลี่ยนเป็น .workbook
#template = pd.read_excel('netsuite_template.xlsx' , sheet = ['template'])

print('transaction record : %i' % len(txn))
print('item master record : %i' % len(m_item))

#txn.head()
#m_item.head()

In [ ]:
txn = pd.merge(txn,m_item[['Internal ID','Item Number','Item Name','SKU Number']] , left_on = '国际条码' , right_on = 'SKU Number', how = 'left').drop('SKU Number' , axis  = 1)

#pyhon จะเอา index ซ้ำมาจึงต้อง drop ออก

#txn.head()

In [ ]:
#sku_rows
df = pd.DataFrame()
#dynamic_values
df[['External ID','Date','NS Internal ID of Item','Item Code','Item Name','Unit Price (Inc. Vat)']] = txn[['订单号' ,'订单支付时间','Internal ID','Item Number','Item Name','商品售价']]
df['Document No'] = df['External ID']
df['Unit Price (Exc VAT)'] = (df['Unit Price (Inc. Vat)'] /1.07).round(2)
df['Amount'] = (df['Unit Price (Exc VAT)']).round(2)
df['VAT Amount'] = (df['Unit Price (Exc VAT)']*0.07).round(2)
df['Gross Amount'] = df['Unit Price (Inc. Vat)']
df['Quantity'] = 1
df['NS Internal ID of Tax Code'] = 7
df['Tax Code'] = 'VAT_TH:VAT 7%'
#static_values
df['Tax Rate'] = "7%"
df['TH Item Display Type'] = ""
df['Sales Payment Method'] = ""
df["Do Not Gen Doc No."] =  "TRUE"
df["Currency"] =  "Thai Baht"
df["Location"] =  "Nexter Digital : BingoBox"
df["Main Location"] =  "Nexter Digital : BingoBox"
df["Subsidiary"] =  "Nexter Digital"
df["TH Tax Type"] =  "Goods"
df["TH Voucher Type"] =  "Cash Sales (ใบกำกับภาษีอย่างย่อ)"
df["Customer Name"] =  "C001368 BingoBox Default"
df["Department"] =  "02009 BingoBox"
df["NS Internal ID of Customer"] =  "28122"
df["NS Internal ID of Department"] =  "311"
df["NS Internal ID of Location"] =  "164"
df["NS Internal ID of Main Location"] =  "164"
df["NS Internal ID of Subsidiary"] =  "14"
df["Exchange Rate"] =  "1"
df["NS Internal ID of TH Tax Type"] =  "1"
df["1. TH Customer's Name"] =  ""
df["2. TH Customer's Branch"] =  ""
df["3. TH Customer's VAT Registration NO"] =  ""
df["Class"] =  ""
df["Description"] =  ""
df["Item Category"] =  ""
df["Memo"] =  ""
df["NS Internal ID of Class"] =  ""
df["NS Internal ID of Item Category"] =  ""
df["NS Internal ID of Item Type"] =  ""
df["NS Internal ID of Sales Channel"] =  ""
df["POS Document No"] =  ""
df["Project"] =  ""
df["Ref No"] =  ""
df["Sales Channel"] =  ""
df["Sales Rep"] =  ""
df["Tax Invoice No"] =  ""
df["WHT Item"] =  ""
#df.head(100)

In [ ]:
#summary_row
sum_df = []
for i in df['External ID'].drop_duplicates() :
    group_transaction = df[df['External ID'] == i]
    amt1 = (group_transaction['Unit Price (Inc. Vat)'].sum())*-1
    amt2 = (amt1/1.07).round(2)
    amt3 = amt2*0.07*-1
    first = group_transaction.iloc[0,:]
    date = first['Date']
    #ext_id = group_transaction['External ID']
    #print(date)
    #print(len(df[df['External ID'] == i]) , i)
    sum_df.append({'External ID': i,
                  'Unit Price (Inc. Vat)': amt1 , 'Unit Price (Exc VAT)' :amt2  , 'Gross Amount' : amt1 , 'Amount' :amt2,
                   'TH Item Display Type' : 'Payment','Tax Code' : 'VAT_TH:NON VAT','Quantity' : '','Tax Rate' : "0%" ,'Sales Payment Method' : '7',
                'NS Internal ID of Tax Code' : '10','NS Internal ID of Item' : '5627','Item Code' : 'Bank transfer Kbank (ND)',
                   'Item Name' : 'Bank transfer Kbank (ND)' ,'Date' : date , 'Document No' : i})
summary_row = pd.DataFrame(sum_df)
summary_row['VAT Amount'] = 0
summary_row['Customer Name'] = 'C001368 BingoBox Default'
summary_row['TH Voucher Type'] = 'Cash Sales (ใบกำกับภาษีอย่างย่อ)'
summary_row['TH Tax Type'] = 'Goods'
summary_row['Subsidiary'] = 'Nexter Digital'
summary_row['Location'] = 'Nexter Digital : BingoBox'
summary_row['Main Location'] = 'Nexter Digital : BingoBox'
summary_row['Currency'] = 'Thai Baht'
summary_row['Do Not Gen Doc No.'] = 'TRUE'
summary_row['Exchange Rate'] = '1'
summary_row['NS Internal ID of TH Tax Type'] = '1'
summary_row['NS Internal ID of Subsidiary'] = '14'
summary_row['NS Internal ID of Location'] = '164'
summary_row['NS Internal ID of Main Location'] = '164'
summary_row['NS Internal ID of Department'] = '311'
summary_row['NS Internal ID of Customer'] = '28122'
summary_row['Department'] = '02009 BingoBox'
summary_row["1. TH Customer's Name"] = ''
summary_row["2. TH Customer's Branch"] = ''
summary_row["3. TH Customer's VAT Registration NO"] = ''
summary_row['Class'] = ''
summary_row['Description'] = ''
summary_row['Item Category'] = ''
summary_row['Memo'] = ''
summary_row['NS Internal ID of Class'] = ''
summary_row['NS Internal ID of Item Category'] = ''
summary_row['NS Internal ID of Item Type'] = ''
summary_row['NS Internal ID of Sales Channel'] = ''
summary_row['POS Document No'] = ''
summary_row['Project'] = ''
summary_row['Ref No'] = ''
summary_row['Sales Channel'] = ''
summary_row['Sales Rep'] = ''
summary_row['Tax Invoice No'] = ''
summary_row['WHT Item'] = ''
#summary_row.head()

In [ ]:
final = pd.DataFrame()
for i in df['External ID'].drop_duplicates() :
    group_tran = df[df['External ID'] == i]
    summary_tran = summary_row[summary_row['External ID'] == i]
    a_invoice = pd.concat([group_tran, summary_tran])
    #print(a_invoice)
    #print(group_tran)
    final = pd.concat([final,a_invoice])

final = final[['External ID', 'Date', 'Do Not Gen Doc No.', 'TH Voucher Type',
       'Document No', 'Tax Invoice No', 'NS Internal ID of Customer',
       'Customer Name', 'NS Internal ID of Subsidiary', 'Subsidiary',
       'NS Internal ID of Main Location', 'Main Location',
       'NS Internal ID of Sales Channel', 'Sales Channel', 'Project',
       'Sales Rep', 'Currency', 'Exchange Rate', 'Ref No', 'Memo',
       'NS Internal ID of TH Tax Type', 'TH Tax Type', "1. TH Customer's Name",
       "2. TH Customer's Branch", "3. TH Customer's VAT Registration NO",
       'NS Internal ID of Item', 'Item Code', 'Item Name', 'Description',
       'NS Internal ID of Location', 'Location', 'NS Internal ID of Class',
       'Class', 'NS Internal ID of Department', 'Department',
       'NS Internal ID of Item Type', 'NS Internal ID of Item Category',
       'Item Category', 'Quantity', 'Unit Price (Inc. Vat)',
       'Unit Price (Exc VAT)', 'Amount', 'NS Internal ID of Tax Code',
       'Tax Code', 'Tax Rate', 'VAT Amount', 'Gross Amount', 'POS Document No',
       'TH Item Display Type', 'Sales Payment Method', 'WHT Item']].reset_index(drop = True)

final.head()

In [ ]:
final.to_excel('netsuite.xlsx' , index = False)
final.to_csv('netsuite.csv' , index = False)